In [9]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import pandas as pd
import pyodbc
import numpy as np

# ============================================================================
# CONFIGURATION DES CHEMINS
# ============================================================================
sol_path = r"C:\Users\pc\Desktop\DM\datasets\sol"
mdb_path = os.path.join(sol_path, "HWSD2.mdb")
bil_path = os.path.join(sol_path, "HWSD2.bil")
shp_path = r"C:\Users\pc\Desktop\DM\datasets\shapefiles\algerie_tunisie.shp"
output_dir = os.path.join(sol_path, "output")

os.makedirs(output_dir, exist_ok=True)

print("=" * 80)
print("EXTRACTION DONNÉES DE SOL HWSD2 - FORMAT POINTS (LAT/LON)")
print("=" * 80)

# ============================================================================
# ÉTAPE 1 : CHARGER LE SHAPEFILE
# ============================================================================
print("\n[1/6] Chargement du shapefile...")

# Essayer plusieurs méthodes pour éviter les problèmes numpy
try:
    import fiona
    with fiona.open(shp_path) as src:
        geoms = [feature['geometry'] for feature in src]
        crs = src.crs
    
    # Créer un GeoDataFrame manuellement
    import shapely.geometry
    from shapely.geometry import shape
    
    geometries = [shape(geom) for geom in geoms]
    gdf_zone = gpd.GeoDataFrame({'geometry': geometries}, crs=crs)
    
    print(f"   ✓ Shapefile chargé : {len(gdf_zone)} entités")
    print(f"   ✓ CRS : {gdf_zone.crs}")
    
except Exception as e1:
    print(f"   ⚠ Méthode 1 échouée : {e1}")
    print("   🔄 Tentative méthode alternative...")
    
    try:
        # Méthode alternative sans geopandas
        gdf_zone = gpd.read_file(shp_path, engine='pyogrio')
        print(f"   ✓ Shapefile chargé : {len(gdf_zone)} entités")
        print(f"   ✓ CRS : {gdf_zone.crs}")
    except Exception as e2:
        print(f"   ✗ Erreur : {e2}")
        raise

# ============================================================================
# ÉTAPE 2 : EXTRAIRE LES DONNÉES D1 DE LA BASE MDB
# ============================================================================
print("\n[2/6] Extraction des données de HWSD2.mdb (LAYER = D1)...")

conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    f'DBQ={mdb_path};'
)

try:
    conn = pyodbc.connect(conn_str)
    print("   ✓ Connexion à la base de données établie")
    
    colonnes = [
        "HWSD2_SMU_ID",
        "COARSE", "SAND", "SILT", "CLAY", 
        "TEXTURE_USDA", "TEXTURE_SOTER", 
        "BULK", "REF_BULK", "ORG_CARBON", "PH_WATER",
        "TOTAL_N", "CN_RATIO", "CEC_SOIL", "CEC_CLAY", 
        "CEC_EFF", "TEB", "BSAT", "ALUM_SAT", "ESP", 
        "TCARBON_EQ", "GYPSUM", "ELEC_COND"
    ]
    
    query = f"""
    SELECT {', '.join(colonnes)}
    FROM HWSD2_LAYERS
    WHERE LAYER = 'D1'
    """
    
    df_sol_D1 = pd.read_sql(query, conn)
    conn.close()
    
    print(f"   ✓ Données extraites : {len(df_sol_D1)} enregistrements (couche D1)")
    print(f"   ✓ Features extraites : {len(colonnes)-1} attributs")
    
except Exception as e:
    print(f"   ✗ Erreur : {e}")
    raise

# ============================================================================
# ÉTAPE 3 : CLIPPER LE RASTER HWSD2.bil
# ============================================================================
print("\n[3/6] Clippage du raster HWSD2.bil...")

with rasterio.open(bil_path) as src:
    print(f"   ✓ Raster ouvert : {src.width} x {src.height} pixels")
    print(f"   ✓ CRS raster : {src.crs}")
    
    if gdf_zone.crs != src.crs:
        print(f"   ⚠ Reprojection du shapefile vers {src.crs}")
        gdf_zone = gdf_zone.to_crs(src.crs)
    
    shapes_geom = [feature["geometry"] for feature in gdf_zone.__geo_interface__["features"]]
    out_image, out_transform = mask(src, shapes_geom, crop=True, nodata=src.nodata)
    out_meta = src.meta.copy()
    
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "compress": "lzw"
    })
    
    raster_clip_path = os.path.join(output_dir, "HWSD2_clipped.tif")
    with rasterio.open(raster_clip_path, "w", **out_meta) as dest:
        dest.write(out_image)
    
    print(f"   ✓ Raster clippé sauvegardé")

# ============================================================================
# ÉTAPE 4 : EXTRAIRE LES POINTS (LONGITUDE, LATITUDE) DU RASTER
# ============================================================================
print("\n[4/6] Extraction des points (longitude, latitude) du raster...")

# Paramètres de filtrage et échantillonnage
LATITUDE_MIN = 32.0   # Garder uniquement le Nord (> 32°N)
SAMPLING_STEP = 10    # ÉTAPE 1: Prendre 1 pixel tous les N pixels (spatial uniforme)

print(f"   📍 Filtrage : latitude > {LATITUDE_MIN}°N (zone Nord)")
print(f"   🎯 Échantillonnage : 1 point tous les {SAMPLING_STEP} pixels")

with rasterio.open(raster_clip_path) as src:
    image = src.read(1)
    transform = src.transform
    
    # Créer les listes pour stocker les données
    longitudes = []
    latitudes = []
    hwsd_ids = []
    
    # Parcourir chaque pixel non-nul
    rows, cols = np.where(image != src.nodata)
    
    print(f"   ⏳ Extraction de {len(rows):,} pixels valides...")
    
    # Compteurs pour statistiques
    total_pixels = len(rows)
    pixels_sampled = 0
    pixels_north = 0
    
    for idx, (row, col) in enumerate(zip(rows, cols)):
        # ÉCHANTILLONNAGE : prendre 1 pixel tous les SAMPLING_STEP
        if idx % SAMPLING_STEP != 0:
            continue
        
        pixels_sampled += 1
        
        # Récupérer la valeur HWSD2_SMU_ID
        hwsd_id = int(image[row, col])
        
        # Convertir les coordonnées pixel en coordonnées géographiques
        lon, lat = rasterio.transform.xy(transform, row, col, offset='center')
        
        # FILTRAGE : garder uniquement latitude > LATITUDE_MIN
        if lat < LATITUDE_MIN:
            continue
        
        pixels_north += 1
        
        longitudes.append(lon)
        latitudes.append(lat)
        hwsd_ids.append(hwsd_id)
    
    print(f"   ✓ Pixels totaux           : {total_pixels:,}")
    print(f"   ✓ Après échantillonnage   : {pixels_sampled:,}")
    print(f"   ✓ Points Nord (lat>{LATITUDE_MIN}°) : {pixels_north:,}")
    print(f"   ✓ Taux de réduction       : {(1 - pixels_north/total_pixels)*100:.1f}%")

# ============================================================================
# ÉTAPE 5 : ÉCHANTILLONNAGE ALÉATOIRE FINAL
# ============================================================================
print("\n[5/6] Échantillonnage aléatoire final...")

# Nombre final de points souhaités (AJUSTABLE)
FINAL_SAMPLE_SIZE = 25000  # Tu peux changer : 20000, 25000, 30000, etc.

# Créer DataFrame temporaire
df_points_temp = pd.DataFrame({
    'longitude': longitudes,
    'latitude': latitudes,
    'HWSD2_SMU_ID': hwsd_ids
})

print(f"   📊 Points avant échantillonnage final : {len(df_points_temp):,}")

# Si on a plus de points que souhaité, échantillonner
if len(df_points_temp) > FINAL_SAMPLE_SIZE:
    print(f"   🎲 Échantillonnage aléatoire vers {FINAL_SAMPLE_SIZE:,} points...")
    
    # Échantillonnage aléatoire simple avec seed fixe (reproductible)
    df_points = df_points_temp.sample(n=FINAL_SAMPLE_SIZE, random_state=42).reset_index(drop=True)
    
    print(f"   ✓ Échantillonnage terminé : {len(df_points):,} points")
    print(f"   ✓ Types de sol uniques    : {df_points['HWSD2_SMU_ID'].nunique():,}")
    print(f"   ✓ Réduction totale        : {(1 - len(df_points)/total_pixels)*100:.2f}%")
else:
    df_points = df_points_temp.copy()
    print(f"   ✓ Pas besoin d'échantillonnage (déjà < {FINAL_SAMPLE_SIZE:,} points)")

print(f"   ✓ Dataset final : {len(df_points):,} points")

# ============================================================================
# ÉTAPE 6 : JOINTURE AVEC LES ATTRIBUTS D1
# ============================================================================
print("\n[6/6] Jointure avec les attributs D1...")

# Jointure sur HWSD2_SMU_ID
df_final = df_points.merge(df_sol_D1, on='HWSD2_SMU_ID', how='left')

print(f"   ⚠ Lignes après jointure   : {len(df_final):,}")

# PROBLÈME : Certains HWSD2_SMU_ID ont plusieurs enregistrements dans D1
# SOLUTION : Garder uniquement la première occurrence (ou faire une agrégation)
n_before = len(df_final)
df_final = df_final.drop_duplicates(subset=['longitude', 'latitude'], keep='first')
n_after = len(df_final)

if n_before != n_after:
    print(f"   🔧 Dédoublonnage effectué : {n_before:,} → {n_after:,} lignes")
    print(f"   ✓ {n_before - n_after:,} doublons supprimés")

# Supprimer HWSD2_SMU_ID (pas demandé dans le résultat final)
df_final = df_final.drop(columns=['HWSD2_SMU_ID'])

print(f"   ✓ Jointure effectuée : {len(df_final):,} points uniques")
print(f"   ✓ Colonnes finales : {len(df_final.columns)} (longitude, latitude + {len(df_final.columns)-2} attributs)")

# Vérifier les valeurs manquantes
nb_manquants = df_final['SAND'].isna().sum()
if nb_manquants > 0:
    print(f"   ⚠ {nb_manquants} points sans attributs D1")

# Réorganiser les colonnes : longitude, latitude en premier
cols = ['longitude', 'latitude'] + [col for col in df_final.columns if col not in ['longitude', 'latitude']]
df_final = df_final[cols]

# ============================================================================
# SAUVEGARDE DES RÉSULTATS
# ============================================================================
print("\n[Sauvegarde] Exportation des résultats...")

# Sauvegarder uniquement le CSV
csv_output = os.path.join(output_dir, "HWSD2_D1_points.csv")
df_final.to_csv(csv_output, index=False, encoding='utf-8-sig')
print(f"   ✓ CSV sauvegardé : {csv_output}")
print(f"   ✓ Taille du fichier : {len(df_final):,} lignes × {len(df_final.columns)} colonnes")

# ============================================================================
# STATISTIQUES FINALES
# ============================================================================
print("\n" + "=" * 80)
print("RÉSUMÉ DES DONNÉES (Couche D1 - 0-20cm de profondeur)")
print("=" * 80)

print(f"\n📍 Zone d'étude : {len(df_final):,} points")
print(f"📊 Features extraites : {len(df_final.columns)-2} attributs")

print("\n📋 Aperçu des premières lignes :")
print(df_final.head(10).to_string(index=False))

print("\n🔬 Liste des features extraites :")
for col in df_final.columns:
    if col not in ['longitude', 'latitude']:
        print(f"   - {col}")

print("\n🌾 Statistiques des propriétés numériques :\n")
proprietes = ['COARSE', 'SAND', 'SILT', 'CLAY', 'BULK', 'REF_BULK', 
              'ORG_CARBON', 'PH_WATER', 'TOTAL_N', 'CN_RATIO', 
              'CEC_SOIL', 'CEC_CLAY', 'CEC_EFF', 'TEB', 'BSAT', 
              'ALUM_SAT', 'ESP', 'TCARBON_EQ', 'GYPSUM', 'ELEC_COND']

for prop in proprietes:
    if prop in df_final.columns:
        valeurs = df_final[prop].dropna()
        if len(valeurs) > 0:
            print(f"   {prop:15s} : min={valeurs.min():8.2f} | "
                  f"max={valeurs.max():8.2f} | mean={valeurs.mean():8.2f}")

# Statistiques sur les coordonnées
print("\n🗺️ Étendue géographique :")
print(f"   Longitude : {df_final['longitude'].min():.4f}° à {df_final['longitude'].max():.4f}°")
print(f"   Latitude  : {df_final['latitude'].min():.4f}° à {df_final['latitude'].max():.4f}°")
print(f"   Zone      : NORD uniquement (lat > {LATITUDE_MIN}°N)")

print("\n✅ Traitement terminé avec succès !")
print(f"\n📁 Fichier de sortie : {csv_output}")
print("\n💡 RÉSUMÉ DU TRAITEMENT :")
print(f"   • Filtrage géographique : latitude > {LATITUDE_MIN}°N")
print(f"   • Échantillonnage spatial : 1 point/{SAMPLING_STEP} pixels")
print(f"   • Échantillonnage aléatoire : max {FINAL_SAMPLE_SIZE:,} points")
print("   • Format : longitude, latitude + attributs de sol")
print("   • Prêt pour fusion avec fire data et machine learning !")

EXTRACTION DONNÉES DE SOL HWSD2 - FORMAT POINTS (LAT/LON)

[1/6] Chargement du shapefile...
   ✓ Shapefile chargé : 2 entités
   ✓ CRS : GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]

[2/6] Extraction des données de HWSD2.mdb (LAYER = D1)...
   ✓ Connexion à la base de données établie


C:\Users\pc\AppData\Local\Temp\ipykernel_12756\3407815944.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sol_D1 = pd.read_sql(query, conn)


   ✓ Données extraites : 58405 enregistrements (couche D1)
   ✓ Features extraites : 22 attributs

[3/6] Clippage du raster HWSD2.bil...
   ✓ Raster ouvert : 43200 x 21600 pixels
   ✓ CRS raster : OGC:CRS84
   ⚠ Reprojection du shapefile vers OGC:CRS84
   ✓ Raster clippé sauvegardé

[4/6] Extraction des points (longitude, latitude) du raster...
   📍 Filtrage : latitude > 32.0°N (zone Nord)
   🎯 Échantillonnage : 1 point tous les 10 pixels
   ⏳ Extraction de 3,295,088 pixels valides...
   ✓ Pixels totaux           : 3,295,088
   ✓ Après échantillonnage   : 329,509
   ✓ Points Nord (lat>32.0°) : 83,845
   ✓ Taux de réduction       : 97.5%

[5/6] Échantillonnage aléatoire final...
   📊 Points avant échantillonnage final : 83,845
   🎲 Échantillonnage aléatoire vers 25,000 points...
   ✓ Échantillonnage terminé : 25,000 points
   ✓ Types de sol uniques    : 333
   ✓ Réduction totale        : 99.24%
   ✓ Dataset final : 25,000 points

[6/6] Jointure avec les attributs D1...
   ⚠ Lignes après

In [6]:
# ============================================================================
# CODE COMPLET - VISUALISATIONS POUR HWSD2_D1_points
# ============================================================================
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

print("\n" + "=" * 80)
print("🚀 CHARGEMENT ET VISUALISATION DES DONNÉES")
print("=" * 80)

# ============================================================================
# ÉTAPE 1 : CHARGEMENT DES DONNÉES
# ============================================================================
print("\n[Étape 1/7] Chargement des données...")

# Définir les chemins
base_dir = r"C:\Users\pc\Desktop\DM\datasets\sol\output"
output_dir = os.path.join(base_dir, "output")
fichier_csv = os.path.join(base_dir, "HWSD2_D1_points.csv")

# Créer le dossier de sortie
os.makedirs(output_dir, exist_ok=True)

# Charger les données
try:
    df_final = pd.read_csv(fichier_csv)
    print(f"   ✅ Données chargées: {df_final.shape[0]:,} lignes × {df_final.shape[1]} colonnes")
    print(f"   ✅ Fichier: {fichier_csv}")
except FileNotFoundError:
    print(f"   ❌ ERREUR: Fichier non trouvé!")
    print(f"   📂 Chemin recherché: {fichier_csv}")
    print("\n💡 Vérifiez que le fichier existe à cet emplacement")
    raise
except Exception as e:
    print(f"   ❌ ERREUR lors du chargement: {e}")
    raise

# ============================================================================
# ÉTAPE 2 : CONFIGURATION
# ============================================================================
print("\n[Étape 2/7] Configuration...")

# Créer le dossier de visualisations
viz_dir = os.path.join(output_dir, "visualisations")
os.makedirs(viz_dir, exist_ok=True)

# Configuration matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

print(f"   ✅ Dossier de sortie: {viz_dir}")
print(f"   ✅ Configuration terminée")

# ============================================================================
# ÉTAPE 3 : DASHBOARD DES VALEURS MANQUANTES
# ============================================================================
print("\n[Étape 3/7] Dashboard des valeurs manquantes...")

all_cols = [col for col in df_final.columns if col not in ['longitude', 'latitude']]
missing_df = pd.DataFrame({
    'Variable': all_cols,
    'Valeurs manquantes': [df_final[col].isna().sum() for col in all_cols],
    'Pourcentage': [(df_final[col].isna().sum() / len(df_final)) * 100 for col in all_cols]
}).set_index('Variable').sort_values('Pourcentage', ascending=False)

try:
    fig = plt.figure(figsize=(16, max(10, len(all_cols) * 0.4)))
    gs = fig.add_gridspec(1, 2, width_ratios=[2, 1])
    fig.suptitle('DASHBOARD DES VALEURS MANQUANTES', fontsize=16, fontweight='bold')

    # Barplot
    ax1 = fig.add_subplot(gs[0])
    colors = ['#d73027' if x > 50 else '#fee090' if x > 10 else '#91cf60' 
              for x in missing_df['Pourcentage']]
    
    bars = ax1.barh(range(len(missing_df)), missing_df['Pourcentage'], 
                    color=colors, edgecolor='black', linewidth=1.2)
    ax1.set_yticks(range(len(missing_df)))
    ax1.set_yticklabels(missing_df.index, fontsize=9)
    ax1.set_xlabel('Pourcentage de valeurs manquantes (%)', fontsize=12, fontweight='bold')
    ax1.set_title('Valeurs manquantes par variable', fontsize=14, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    
    for i, (bar, val) in enumerate(zip(bars, missing_df['Pourcentage'])):
        if val > 0:
            ax1.text(val + 0.5, i, f'{val:.1f}%', va='center', fontsize=8, fontweight='bold')
    
    ax1.axvline(10, color='orange', linestyle='--', linewidth=2, label='10%')
    ax1.axvline(50, color='red', linestyle='--', linewidth=2, label='50%')
    ax1.legend(fontsize=10)
    
    # Stats
    ax2 = fig.add_subplot(gs[1])
    ax2.axis('off')
    
    total_cells = len(df_final) * len(all_cols)
    total_missing = int(missing_df['Valeurs manquantes'].sum())
    pct_global = (total_missing / total_cells) * 100
    
    stats_text = f"""STATISTIQUES GLOBALES

📊 Observations: {len(df_final):,}
📊 Variables: {len(all_cols)}
📊 Cellules: {total_cells:,}

❌ Manquantes: {total_missing:,}
📉 Taux: {pct_global:.2f}%
✅ Complétude: {100-pct_global:.2f}%

TOP 5 MANQUANTS:"""
    
    for i, (idx, row) in enumerate(missing_df.head(5).iterrows(), 1):
        if row['Valeurs manquantes'] > 0:
            stats_text += f"\n{i}. {idx}: {row['Pourcentage']:.1f}%"
    
    ax2.text(0.05, 0.5, stats_text, fontsize=10, va='center', family='monospace',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, '01_valeurs_manquantes.png'), dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   ✅ 01_valeurs_manquantes.png")
except Exception as e:
    print(f"   ❌ Erreur: {e}")
    plt.close('all')

# ============================================================================
# ÉTAPE 4 : MATRICE DE CORRÉLATION
# ============================================================================
print("\n[Étape 4/7] Matrice de corrélation...")

try:
    numeric_cols = [col for col in df_final.select_dtypes(include=[np.number]).columns 
                    if col not in ['longitude', 'latitude']]
    numeric_data = df_final[numeric_cols].dropna(thresh=len(df_final)*0.5, axis=1)

    if len(numeric_data.columns) > 1:
        corr_matrix = numeric_data.corr()
        
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))
        fig.suptitle('MATRICE DE CORRÉLATION', fontsize=16, fontweight='bold')
        
        # Heatmap
        im = axes[0].imshow(corr_matrix.values, cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
        
        for i in range(len(corr_matrix)):
            for j in range(len(corr_matrix.columns)):
                if i > j:
                    axes[0].text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                               ha="center", va="center", color="black", fontsize=8)
        
        axes[0].set_xticks(range(len(corr_matrix)))
        axes[0].set_yticks(range(len(corr_matrix)))
        axes[0].set_xticklabels(corr_matrix.columns, rotation=90, ha='right')
        axes[0].set_yticklabels(corr_matrix.columns)
        axes[0].set_title('Corrélation (Pearson)', fontweight='bold')
        plt.colorbar(im, ax=axes[0])
        
        # Top corrélations
        corr_pairs = corr_matrix.unstack()
        corr_pairs = corr_pairs[corr_pairs < 1].sort_values(ascending=False)
        top_corr = corr_pairs.head(15)
        
        colors = ['#d73027' if abs(x) > 0.7 else '#fee090' if abs(x) > 0.4 else '#91cf60' 
                  for x in top_corr.values]
        
        axes[1].barh(range(len(top_corr)), top_corr.values, color=colors, edgecolor='black')
        axes[1].set_yticks(range(len(top_corr)))
        axes[1].set_yticklabels([f"{p[0]} ↔ {p[1]}" for p in top_corr.index], fontsize=9)
        axes[1].set_xlabel('Corrélation', fontweight='bold')
        axes[1].set_title('Top 15 corrélations', fontweight='bold')
        axes[1].axvline(0.7, color='red', linestyle='--', linewidth=1, label='Forte (0.7)')
        axes[1].axvline(-0.7, color='red', linestyle='--', linewidth=1)
        axes[1].axvline(0.4, color='orange', linestyle='--', linewidth=1, label='Modérée (0.4)')
        axes[1].axvline(-0.4, color='orange', linestyle='--', linewidth=1)
        axes[1].legend()
        axes[1].grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, '02_correlations.png'), dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅ 02_correlations.png")
    else:
        print(f"   ⚠️  Pas assez de variables numériques")
except Exception as e:
    print(f"   ❌ Erreur: {e}")
    plt.close('all')

# ============================================================================
# ÉTAPE 5 : DISTRIBUTIONS
# ============================================================================
print("\n[Étape 5/7] Histogrammes des distributions...")

try:
    main_props = ['SAND', 'SILT', 'CLAY', 'BULK', 'ORG_CARBON', 'PH_WATER', 
                  'CEC_SOIL', 'TOTAL_N', 'CN_RATIO', 'COARSE', 'REF_BULK', 'TEB']
    available_props = [p for p in main_props if p in df_final.columns]

    if available_props:
        n_cols = 3
        n_rows = (len(available_props) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
        fig.suptitle('DISTRIBUTIONS DES PROPRIÉTÉS DU SOL', fontsize=16, fontweight='bold')
        axes = axes.flatten() if len(available_props) > 1 else [axes]
        
        for idx, prop in enumerate(available_props):
            ax = axes[idx]
            data = df_final[prop].dropna()
            
            if len(data) > 0:
                ax.hist(data.values, bins=30, alpha=0.7, color='steelblue', edgecolor='black')
                
                mean_val = float(data.mean())
                median_val = float(data.median())
                ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, 
                          label=f'Moy: {mean_val:.2f}')
                ax.axvline(median_val, color='green', linestyle='--', linewidth=2,
                          label=f'Méd: {median_val:.2f}')
                
                ax.set_xlabel(prop, fontweight='bold')
                ax.set_ylabel('Fréquence', fontweight='bold')
                ax.set_title(f'{prop}', fontweight='bold')
                ax.legend(fontsize=8)
                ax.grid(alpha=0.3)
                
                std = float(data.std())
                n = len(data)
                skew = float(((data - mean_val) ** 3).sum() / (n * std ** 3)) if std > 0 else 0
                
                stats = f'n={n}\nstd={std:.2f}\nskew={skew:.2f}'
                ax.text(0.98, 0.97, stats, transform=ax.transAxes, va='top', ha='right',
                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5), fontsize=9)
        
        for idx in range(len(available_props), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, '03_distributions.png'), dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅ 03_distributions.png")
except Exception as e:
    print(f"   ❌ Erreur: {e}")
    plt.close('all')

# ============================================================================
# ÉTAPE 6 : BOXPLOTS
# ============================================================================
print("\n[Étape 6/7] Boxplots...")

try:
    if available_props:
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
        fig.suptitle('BOXPLOTS - DÉTECTION DES OUTLIERS', fontsize=16, fontweight='bold')
        axes = axes.flatten() if len(available_props) > 1 else [axes]
        
        for idx, prop in enumerate(available_props):
            ax = axes[idx]
            data = df_final[prop].dropna()
            
            if len(data) > 0:
                bp = ax.boxplot([data.values], patch_artist=True, widths=0.6,
                               boxprops=dict(facecolor='lightblue', edgecolor='black'),
                               medianprops=dict(color='red', linewidth=2))
                
                Q1 = float(data.quantile(0.25))
                Q3 = float(data.quantile(0.75))
                IQR = Q3 - Q1
                
                ax.set_ylabel(prop, fontweight='bold')
                ax.set_title(f'{prop}', fontweight='bold')
                ax.set_xticks([])
                ax.grid(axis='y', alpha=0.3)
                
                stats = (f'Min: {float(data.min()):.2f}\nQ1: {Q1:.2f}\n'
                        f'Méd: {float(data.median()):.2f}\nQ3: {Q3:.2f}\n'
                        f'Max: {float(data.max()):.2f}\nIQR: {IQR:.2f}')
                ax.text(1.15, 0.5, stats, transform=ax.transAxes, va='center',
                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5), fontsize=9)
        
        for idx in range(len(available_props), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, '04_boxplots.png'), dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅ 04_boxplots.png")
except Exception as e:
    print(f"   ❌ Erreur: {e}")
    plt.close('all')

# ============================================================================
# ÉTAPE 7 : QQ-PLOTS (comparaison entre variables corrélées)
# ============================================================================
print("\n[Étape 7/7] QQ-Plots pour comparer les distributions...")

try:
    if len(numeric_data.columns) > 1 and 'corr_matrix' in locals():
        # Trouver les 6 paires les plus corrélées
        corr_pairs_qq = corr_matrix.unstack()
        corr_pairs_qq = corr_pairs_qq[corr_pairs_qq < 1]
        top_pairs = corr_pairs_qq.abs().sort_values(ascending=False).head(6).index.tolist()
        
        n_pairs = len(top_pairs)
        n_cols_qq = 3
        n_rows_qq = (n_pairs + n_cols_qq - 1) // n_cols_qq
        
        fig, axes = plt.subplots(n_rows_qq, n_cols_qq, figsize=(18, 5*n_rows_qq))
        fig.suptitle('QQ-PLOTS - COMPARAISON DES DISTRIBUTIONS', fontsize=16, fontweight='bold')
        axes = axes.flatten() if n_pairs > 1 else [axes]
        
        for idx, (var1, var2) in enumerate(top_pairs):
            ax = axes[idx]
            
            data1 = df_final[var1].dropna()
            data2 = df_final[var2].dropna()
            
            if len(data1) > 10 and len(data2) > 10:
                # Trier les données
                data1_sorted = np.sort(data1.values)
                data2_sorted = np.sort(data2.values)
                
                # Interpoler pour avoir la même longueur
                n = min(len(data1_sorted), len(data2_sorted))
                quantiles1 = np.percentile(data1_sorted, np.linspace(0, 100, n))
                quantiles2 = np.percentile(data2_sorted, np.linspace(0, 100, n))
                
                # Créer le QQ-plot
                ax.scatter(quantiles1, quantiles2, alpha=0.6, s=20, 
                          color='steelblue', edgecolor='black')
                
                # Ligne de référence (identité)
                min_val = min(float(quantiles1.min()), float(quantiles2.min()))
                max_val = max(float(quantiles1.max()), float(quantiles2.max()))
                ax.plot([min_val, max_val], [min_val, max_val], 'r--', 
                       linewidth=2, label='Ligne identité')
                
                ax.set_xlabel(f'{var1}', fontweight='bold')
                ax.set_ylabel(f'{var2}', fontweight='bold')
                ax.set_title(f'QQ-Plot: {var1} vs {var2}', fontweight='bold', pad=10)
                ax.grid(alpha=0.3)
                ax.legend(fontsize=8)
                
                # Afficher le coefficient de corrélation
                corr_val = float(corr_matrix.loc[var1, var2])
                ax.text(0.05, 0.95, f'r = {corr_val:.3f}', transform=ax.transAxes,
                       va='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
                       fontsize=10, fontweight='bold')
        
        # Masquer les axes vides
        for idx in range(n_pairs, len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, '05_qqplots.png'), dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅ 05_qqplots.png")
    else:
        print("   ⚠️  Pas assez de variables pour créer des QQ-plots")
except Exception as e:
    print(f"   ❌ Erreur: {e}")
    import traceback
    traceback.print_exc()
    plt.close('all')

# ============================================================================
# RAPPORT FINAL
# ============================================================================
print("\n" + "=" * 80)
print("✅ VISUALISATIONS TERMINÉES!")
print("=" * 80)

files = sorted([f for f in os.listdir(viz_dir) if f.endswith('.png')])
if files:
    print(f"\n🎉 {len(files)} fichiers générés:\n")
    for f in files:
        size = os.path.getsize(os.path.join(viz_dir, f)) / 1024
        print(f"   ✅ {f} ({size:.1f} KB)")
    print(f"\n📁 Dossier: {viz_dir}")
    print(f"\n💡 Ouvrez le dossier pour voir vos visualisations!")
else:
    print("\n⚠️  Aucun fichier généré - vérifiez les erreurs ci-dessus")

print("\n📊 Liste complète des visualisations:")
print("   1. 01_valeurs_manquantes.png - Dashboard des valeurs manquantes")
print("   2. 02_correlations.png - Matrice de corrélation et top corrélations")
print("   3. 03_distributions.png - Histogrammes des distributions")
print("   4. 04_boxplots.png - Boxplots pour détecter les outliers")
print("   5. 05_qqplots.png - QQ-plots entre variables corrélées")

print("\n" + "=" * 80)


🚀 CHARGEMENT ET VISUALISATION DES DONNÉES

[Étape 1/7] Chargement des données...
   ✅ Données chargées: 25,000 lignes × 24 colonnes
   ✅ Fichier: C:\Users\pc\Desktop\DM\datasets\sol\output\HWSD2_D1_points.csv

[Étape 2/7] Configuration...
   ✅ Dossier de sortie: C:\Users\pc\Desktop\DM\datasets\sol\output\output\visualisations
   ✅ Configuration terminée

[Étape 3/7] Dashboard des valeurs manquantes...
   ✅ 01_valeurs_manquantes.png

[Étape 4/7] Matrice de corrélation...
   ✅ 02_correlations.png

[Étape 5/7] Histogrammes des distributions...
   ✅ 03_distributions.png

[Étape 6/7] Boxplots...
   ✅ 04_boxplots.png

[Étape 7/7] QQ-Plots pour comparer les distributions...
   ✅ 05_qqplots.png

✅ VISUALISATIONS TERMINÉES!

🎉 5 fichiers générés:

   ✅ 01_valeurs_manquantes.png (289.7 KB)
   ✅ 02_correlations.png (767.7 KB)
   ✅ 03_distributions.png (806.9 KB)
   ✅ 04_boxplots.png (627.2 KB)
   ✅ 05_qqplots.png (466.6 KB)

📁 Dossier: C:\Users\pc\Desktop\DM\datasets\sol\output\output\visualisat

In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
print("=" * 80)
print("NETTOYAGE DES DONNÉES DE SOL HWSD2")
print("=" * 80)

# Chemins
sol_path = r"C:\Users\pc\Desktop\DM\datasets\sol"
output_dir = os.path.join(sol_path, "output")
cleaned_dir = os.path.join(output_dir, "cleaned_data")
os.makedirs(cleaned_dir, exist_ok=True)

# Charger les données
print("\n[1/6] Chargement des données...")
# MODIFIÉ : Charger depuis CSV au lieu de GeoPackage
csv_path = os.path.join(output_dir, "HWSD2_D1_points.csv")  # Adaptez le nom du fichier
df = pd.read_csv(csv_path)
print(f"   ✓ Données chargées : {len(df)} enregistrements, {len(df.columns)} colonnes")

# Sauvegarder une copie des données originales
df_original = df.copy()

# ============================================================================
# ÉTAPE 1 : IDENTIFICATION DES COLONNES PAR TYPE
# ============================================================================
print("\n[2/6] Identification des types de colonnes...")

# MODIFIÉ : Colonnes géographiques au lieu de geometry
geo_cols = ['longitude', 'latitude']
categorical_cols = ['TEXTURE_USDA', 'TEXTURE_SOTER']
numeric_cols = [col for col in df.columns 
                if col not in categorical_cols + geo_cols]

print(f"   ✓ Colonnes numériques : {len(numeric_cols)}")
print(f"   ✓ Colonnes catégorielles : {len(categorical_cols)}")
print(f"   ✓ Colonnes géographiques : {len(geo_cols)}")

# Statistiques initiales
print("\n   📊 Valeurs manquantes avant nettoyage :")
for col in df.columns:
    if col not in geo_cols:
        missing = df[col].isna().sum()
        if missing > 0:
            pct = (missing / len(df)) * 100
            print(f"      {col:20s} : {missing:5d} ({pct:5.1f}%)")

# ============================================================================
# ÉTAPE 2 : TRAITEMENT DES VALEURS ABERRANTES (-9 = valeur manquante)
# ============================================================================
print("\n[3/6] Traitement des valeurs codées -9 (valeurs manquantes)...")

for col in numeric_cols:
    if col in df.columns:
        mask = df[col] == -9
        n_replaced = mask.sum()
        if n_replaced > 0:
            df.loc[mask, col] = np.nan
            print(f"   ✓ {col:20s} : {n_replaced:5d} valeurs -9 remplacées par NaN")

# ============================================================================
# ÉTAPE 3 : IMPUTATION DES VALEURS MANQUANTES AVEC KNN
# ============================================================================
print("\n[4/6] Imputation des valeurs manquantes...")

# 3.1 IMPUTATION DES COLONNES NUMÉRIQUES AVEC KNN
print("\n   🔢 Imputation des colonnes numériques avec KNN...")

numeric_data = df[numeric_cols].copy()
missing_counts = numeric_data.isna().sum()
cols_with_missing = missing_counts[missing_counts > 0].index.tolist()

if len(cols_with_missing) > 0:
    print(f"      ℹ️  {len(cols_with_missing)} colonnes numériques avec valeurs manquantes")
    
    knn_imputer = KNNImputer(n_neighbors=5, weights='distance')
    print("      ⏳ Application de KNN Imputer...")
    numeric_imputed = knn_imputer.fit_transform(numeric_data)
    
    numeric_data_imputed = pd.DataFrame(
        numeric_imputed, 
        columns=numeric_cols,
        index=numeric_data.index
    )
    
    for col in numeric_cols:
        df[col] = numeric_data_imputed[col]
    
    print(f"      ✓ KNN Imputation terminée pour {len(cols_with_missing)} colonnes")
    
    for col in cols_with_missing:
        original_missing = numeric_data[col].isna().sum()
        new_missing = df[col].isna().sum()
        print(f"         {col:20s} : {original_missing} → {new_missing} valeurs manquantes")
else:
    print("      ✓ Aucune valeur manquante dans les colonnes numériques")

# 3.2 IMPUTATION DES COLONNES CATÉGORIELLES
print("\n   🏷️  Imputation des colonnes catégorielles avec KNN...")

categorical_encoders_temp = {}

for col in categorical_cols:
    if col in df.columns:
        missing_count = df[col].isna().sum()
        
        if missing_count > 0:
            print(f"\n      📊 Traitement de {col}...")
            print(f"         Valeurs manquantes avant : {missing_count} ({missing_count/len(df)*100:.1f}%)")
            
            le_temp = LabelEncoder()
            col_data = df[col].copy()
            col_data = col_data.astype(str)
            col_data = col_data.replace(['nan', 'None', ''], np.nan)
            
            mask_not_missing = col_data.notna()
            values_not_missing = col_data[mask_not_missing]
            encoded_values = le_temp.fit_transform(values_not_missing)
            
            col_encoded = np.full(len(col_data), np.nan)
            col_encoded[mask_not_missing] = encoded_values
            
            temp_df = df[numeric_cols].copy()
            temp_df[f'{col}_encoded_temp'] = col_encoded
            
            knn_cat = KNNImputer(n_neighbors=5, weights='distance')
            temp_imputed = knn_cat.fit_transform(temp_df[[f'{col}_encoded_temp']])
            
            imputed_encoded = np.round(temp_imputed[:, 0]).astype(int)
            imputed_categories = le_temp.inverse_transform(imputed_encoded)
            
            df[col] = imputed_categories
            
            new_missing = df[col].isna().sum()
            print(f"         ✓ Valeurs manquantes après  : {new_missing} ({new_missing/len(df)*100:.1f}%)")
            
            categorical_encoders_temp[col] = le_temp
        else:
            print(f"\n      ✓ {col} : Aucune valeur manquante")

print("\n   ✅ Imputation complète (numériques + catégorielles avec KNN)")

# ============================================================================
# ÉTAPE 4 : GESTION DES OUTLIERS AVEC TRANSFORMATION LOG
# ============================================================================
print("\n[5/6] Gestion des outliers avec transformation logarithmique...")

def detect_outliers_iqr(data, col, threshold=1.5):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    outliers = ((data[col] < lower_bound) | (data[col] > upper_bound)).sum()
    return outliers, (outliers / len(data)) * 100

outlier_report = []
for col in numeric_cols:
    if col in df.columns:
        n_outliers, pct_outliers = detect_outliers_iqr(df, col)
        if n_outliers > 0:
            outlier_report.append({
                'column': col,
                'n_outliers': n_outliers,
                'pct_outliers': pct_outliers
            })

outlier_df = pd.DataFrame(outlier_report).sort_values('pct_outliers', ascending=False)
cols_to_transform = []

if len(outlier_df) > 0:
    print(f"\n   📊 Colonnes avec outliers (top 10) :")
    for idx, row in outlier_df.head(10).iterrows():
        print(f"      {row['column']:20s} : {row['n_outliers']:5d} outliers ({row['pct_outliers']:5.1f}%)")
    
    cols_to_transform = outlier_df[outlier_df['pct_outliers'] > 5]['column'].tolist()
    
    if len(cols_to_transform) > 0:
        print(f"\n   🔄 Application de log(x+1) sur {len(cols_to_transform)} colonnes...")
        
        for col in cols_to_transform:
            min_val = df[col].min()
            
            if min_val < 0:
                df[col] = df[col] - min_val + 1
                print(f"      ⚠️  {col}: décalage de {-min_val + 1:.2f} appliqué")
            
            df[f'{col}_log'] = np.log1p(df[col])
            print(f"      ✓ {col}: transformation log appliquée → {col}_log")
        
        numeric_cols.extend([f'{col}_log' for col in cols_to_transform])
else:
    print("   ✓ Aucun outlier significatif détecté")

# ============================================================================
# ÉTAPE 5 : ENCODING DES VARIABLES CATÉGORIELLES
# ============================================================================
print("\n[6/6] Encoding des variables catégorielles...")

encoders = {}
encoded_mapping = {}

for col in categorical_cols:
    if col in df.columns:
        print(f"\n   📊 Traitement de {col}...")
        
        df[f'{col}_encoded'] = df[col].copy()
        df[f'{col}_encoded'] = df[f'{col}_encoded'].astype(str)
        df[f'{col}_encoded'] = df[f'{col}_encoded'].replace(['nan', 'None', ''], 'Unknown')
        
        unique_values = df[f'{col}_encoded'].unique()
        print(f"      ℹ️  {len(unique_values)} valeurs uniques détectées")
        
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[f'{col}_encoded'])
        
        encoders[col] = le
        mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        encoded_mapping[col] = mapping
        
        print(f"      ✓ {col}: {len(le.classes_)} catégories encodées")
        print(f"      📋 Aperçu du mapping (5 premières valeurs):")
        for i, (original, encoded) in enumerate(list(mapping.items())[:5]):
            print(f"         {original:30s} → {encoded}")
        
        if len(mapping) > 5:
            print(f"         ... et {len(mapping) - 5} autres catégories")

print("\n   ✅ Encoding des variables catégorielles terminé")

print("\n   🗑️  Suppression des colonnes catégorielles originales...")
for col in categorical_cols:
    if col in df.columns:
        df = df.drop(columns=[col])
        print(f"      ✓ {col} supprimée (remplacée par {col}_encoded)")

# ============================================================================
# ÉTAPE 6 : SUPPRESSION DES COLONNES ORIGINALES TRANSFORMÉES EN LOG
# ============================================================================
print("\n[7/9] Suppression des colonnes originales (remplacées par log)...")

if len(cols_to_transform) > 0:
    for col in cols_to_transform:
        if col in df.columns:
            df = df.drop(columns=[col])
            print(f"   ✓ {col} supprimée (remplacée par {col}_log)")
    print(f"   ✅ {len(cols_to_transform)} colonnes originales supprimées")
else:
    print("   ℹ️  Aucune colonne transformée en log")

# ============================================================================
# ÉTAPE 7 : SUPPRESSION DES COLONNES FORTEMENT CORRÉLÉES
# ============================================================================
print("\n[8/9] Analyse et suppression des colonnes fortement corrélées...")

current_numeric_cols = [col for col in df.columns 
                       if col not in categorical_cols + geo_cols
                       and not col.endswith('_encoded')]

print(f"   ℹ️  Analyse de {len(current_numeric_cols)} colonnes numériques...")

numeric_df = df[current_numeric_cols]
corr_matrix = numeric_df.corr().abs()

correlation_threshold = 0.85

upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

cols_to_drop = set()
correlation_pairs = []

for column in upper_triangle.columns:
    highly_correlated = upper_triangle.index[upper_triangle[column] > correlation_threshold].tolist()
    
    for corr_col in highly_correlated:
        correlation_pairs.append({
            'col1': column,
            'col2': corr_col,
            'correlation': upper_triangle.loc[corr_col, column]
        })
        
        count_col = (upper_triangle[column] > correlation_threshold).sum()
        count_corr = (upper_triangle[corr_col] > correlation_threshold).sum()
        
        if count_col >= count_corr:
            cols_to_drop.add(column)
        else:
            cols_to_drop.add(corr_col)

cols_to_drop = list(cols_to_drop)

print(f"\n   📊 Statistiques de corrélation :")
print(f"      • Colonnes analysées : {len(current_numeric_cols)}")
print(f"      • Paires fortement corrélées (r > {correlation_threshold}) : {len(correlation_pairs)}")
print(f"      • Colonnes identifiées pour suppression : {len(cols_to_drop)}")

if len(correlation_pairs) > 0:
    print(f"\n   📈 Top 10 des paires les plus corrélées :")
    corr_df = pd.DataFrame(correlation_pairs).sort_values('correlation', ascending=False)
    for idx, row in corr_df.head(10).iterrows():
        marker1 = "🗑️ " if row['col1'] in cols_to_drop else "✅"
        marker2 = "🗑️ " if row['col2'] in cols_to_drop else "✅"
        print(f"      {marker1} {row['col1']:20s} ↔ {marker2} {row['col2']:20s} : r = {row['correlation']:.3f}")

if len(cols_to_drop) > 0:
    print(f"\n   🗑️  Suppression de {len(cols_to_drop)} colonnes redondantes :")
    
    for col in sorted(cols_to_drop):
        n_corr = (upper_triangle[col] > correlation_threshold).sum()
        print(f"      • {col:30s} ({n_corr} corrélations fortes)")
    
    cols_actually_dropped = []
    for col in cols_to_drop:
        if col in df.columns:
            df = df.drop(columns=[col])
            cols_actually_dropped.append(col)
    
    print(f"\n   ✅ {len(cols_actually_dropped)} colonnes effectivement supprimées")
    print(f"   📊 Colonnes restantes : {len(df.columns)}")
    
    cols_to_drop = cols_actually_dropped
else:
    print(f"\n   ✅ Aucune colonne fortement corrélée détectée (seuil r > {correlation_threshold})")

# ============================================================================
# ÉTAPE 8 : SAUVEGARDE
# ============================================================================
print("\n[9/9] Sauvegarde des données nettoyées...")

print("\n   📊 Statistiques finales :")
print(f"      Enregistrements : {len(df)}")
print(f"      Colonnes totales : {len(df.columns)}")
print(f"      Colonnes numériques : {len([c for c in df.columns if c not in categorical_cols + geo_cols and not c.endswith('_encoded')])}")
print(f"      Colonnes encodées : {len([c for c in df.columns if c.endswith('_encoded')])}")
print(f"      Colonnes géographiques : {len(geo_cols)}")

remaining_missing = df.drop(columns=geo_cols).isna().sum()
cols_with_missing_final = remaining_missing[remaining_missing > 0]

if len(cols_with_missing_final) > 0:
    print(f"\n   ⚠️  Valeurs manquantes restantes :")
    for col, count in cols_with_missing_final.items():
        pct = (count / len(df)) * 100
        print(f"      {col:20s} : {count:5d} ({pct:5.1f}%)")
else:
    print("\n   ✅ Aucune valeur manquante !")

print("\n   💾 Sauvegarde des fichiers...")

# CSV
csv_cleaned = os.path.join(cleaned_dir, "HWSD2_D1_cleaned.csv")
df.to_csv(csv_cleaned, index=False, encoding='utf-8-sig')
print(f"      ✓ CSV : {csv_cleaned}")

# Excel
try:
    excel_cleaned = os.path.join(cleaned_dir, "HWSD2_D1_cleaned.xlsx")
    df.to_excel(excel_cleaned, index=False, engine='openpyxl')
    print(f"      ✓ Excel : {excel_cleaned}")
except Exception as e:
    print(f"      ⚠️  Excel non créé : {e}")

# Encoders
import pickle
encoders_path = os.path.join(cleaned_dir, "label_encoders.pkl")
with open(encoders_path, 'wb') as f:
    pickle.dump(encoders, f)
print(f"      ✓ Encoders : {encoders_path}")

# Mapping
mapping_path = os.path.join(cleaned_dir, "encoding_mapping.txt")
with open(mapping_path, 'w', encoding='utf-8') as f:
    f.write("MAPPING DES VARIABLES CATÉGORIELLES\n")
    f.write("=" * 80 + "\n\n")
    for col, mapping in encoded_mapping.items():
        f.write(f"{col}:\n")
        f.write(f"  Nombre de catégories: {len(mapping)}\n\n")
        for original, encoded in sorted(mapping.items(), key=lambda x: x[1]):
            f.write(f"   {original:30s} → {encoded}\n")
        f.write("\n" + "-" * 80 + "\n\n")
print(f"      ✓ Mapping : {mapping_path}")

print("\n" + "=" * 80)
print("✅ NETTOYAGE TERMINÉ AVEC SUCCÈS")
print("=" * 80)
print(f"\n📁 Fichiers sauvegardés dans : {cleaned_dir}")
print("\n📊 RÉSUMÉ :")
print(f"   • Valeurs -9 remplacées")
print(f"   • KNN Imputation appliquée")
if len(cols_to_transform) > 0:
    print(f"   • Transformation log sur {len(cols_to_transform)} colonnes")
print(f"   • {len(categorical_cols)} variables encodées")
if len(cols_to_drop) > 0:
    print(f"   • {len(cols_to_drop)} colonnes corrélées supprimées")
print(f"   • Dataset final : {len(df)} × {len(df.columns)}")

NETTOYAGE DES DONNÉES DE SOL HWSD2

[1/6] Chargement des données...
   ✓ Données chargées : 25000 enregistrements, 24 colonnes

[2/6] Identification des types de colonnes...
   ✓ Colonnes numériques : 20
   ✓ Colonnes catégorielles : 2
   ✓ Colonnes géographiques : 2

   📊 Valeurs manquantes avant nettoyage :
      TEXTURE_USDA         :  1269 (  5.1%)
      REF_BULK             :  1269 (  5.1%)

[3/6] Traitement des valeurs codées -9 (valeurs manquantes)...
   ✓ COARSE               :   116 valeurs -9 remplacées par NaN
   ✓ SAND                 :   116 valeurs -9 remplacées par NaN
   ✓ SILT                 :   116 valeurs -9 remplacées par NaN
   ✓ CLAY                 :   116 valeurs -9 remplacées par NaN
   ✓ BULK                 :   116 valeurs -9 remplacées par NaN
   ✓ ORG_CARBON           :   116 valeurs -9 remplacées par NaN
   ✓ PH_WATER             :   116 valeurs -9 remplacées par NaN
   ✓ TOTAL_N              :   116 valeurs -9 remplacées par NaN
   ✓ CN_RATIO            

In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
print("=" * 80)
print("FUSION SPATIALE : FIRE + SOIL (K-NN + IDW)")
print("=" * 80)

# Chemins des fichiers
base_path = r"C:\Users\pc\Desktop\DM\datasets"
soil_path = os.path.join(base_path, "sol\output\cleaned_data", "HWSD2_D1_cleaned.csv")
fire_path = os.path.join(base_path, "fire_data", "fire_nonfire_north_unbalanced.csv")
output_dir = os.path.join(base_path, "merged_fire_soil")

# Paramètres K-NN et IDW
K_NEIGHBORS = 5  # Nombre de voisins pour K-NN
POWER_IDW = 2    # Puissance pour Inverse Distance Weighting
DISTANCE_THRESHOLD = 0.1  # Seuil de distance (degrés) pour considérer un match direct

# Créer le dossier de sortie
os.makedirs(output_dir, exist_ok=True)

# ============================================================================
# ÉTAPE 1 : CHARGER LES DONNÉES
# ============================================================================
print("\n[1/7] Chargement des données...")

print("   📍 Chargement du dataset FIRE...")
df_fire = pd.read_csv(fire_path)
print(f"      ✓ Fire chargé : {len(df_fire):,} points")

print("\n   🌍 Chargement du dataset SOIL...")
df_soil = pd.read_csv(soil_path)
print(f"      ✓ Soil chargé : {len(df_soil):,} points")

# Vérifier les colonnes
print(f"\n   📋 Colonnes FIRE : {df_fire.columns.tolist()[:5]}...")
print(f"   📋 Colonnes SOIL : {df_soil.columns.tolist()[:5]}...")

# ============================================================================
# ÉTAPE 2 : IDENTIFIER COLONNES NUMÉRIQUES VS CATÉGORIELLES
# ============================================================================
print("\n[2/7] Identification des types de colonnes SOIL...")

# Colonnes SOIL à copier (exclure longitude/latitude)
soil_cols_to_copy = [col for col in df_soil.columns 
                     if col not in ['longitude', 'latitude']]

# Identifier les colonnes numériques et catégorielles
numeric_cols = []
categorical_cols = []

for col in soil_cols_to_copy:
    if df_soil[col].dtype in [np.float64, np.float32, np.int64, np.int32, np.int16, np.int8]:
        numeric_cols.append(col)
    else:
        categorical_cols.append(col)

print(f"   📊 Colonnes numériques (IDW)       : {len(numeric_cols)}")
print(f"   📋 Colonnes catégorielles (K-NN)   : {len(categorical_cols)}")

if len(numeric_cols) > 0:
    print(f"      Exemples numériques : {numeric_cols[:5]}")
if len(categorical_cols) > 0:
    print(f"      Exemples catégorielles : {categorical_cols[:5]}")

# ============================================================================
# ÉTAPE 3 : APPLIQUER K-NN POUR TOUS LES POINTS
# ============================================================================
print("\n[3/7] Application K-NN pour tous les points FIRE...")

# Préparer les données pour K-NN
X_train = df_soil[['longitude', 'latitude']].values  # Points SOIL
X_test = df_fire[['longitude', 'latitude']].values   # Points FIRE

print(f"   🔄 Recherche des {K_NEIGHBORS} plus proches points SOIL...")

# K-NN pour trouver les k plus proches points SOIL
knn = NearestNeighbors(n_neighbors=K_NEIGHBORS, metric='euclidean')
knn.fit(X_train)

# Trouver les K plus proches voisins
distances, indices = knn.kneighbors(X_test)

print(f"      ✓ K-NN terminé")
print(f"      • Distance min (1er voisin) : {distances[:, 0].min():.6f}° (~{distances[:, 0].min()*111:.2f} km)")
print(f"      • Distance max (1er voisin) : {distances[:, 0].max():.6f}° (~{distances[:, 0].max()*111:.2f} km)")
print(f"      • Distance moyenne          : {distances[:, 0].mean():.6f}° (~{distances[:, 0].mean()*111:.2f} km)")

# ============================================================================
# ÉTAPE 4 : CLASSIFICATION DES POINTS (DIRECT MATCH VS INTERPOLATION)
# ============================================================================
print(f"\n[4/7] Classification des points (seuil = {DISTANCE_THRESHOLD}°)...")

# Points avec match direct (distance < seuil)
direct_match_mask = distances[:, 0] < DISTANCE_THRESHOLD
n_direct = direct_match_mask.sum()
n_interpolated = len(df_fire) - n_direct

print(f"   📍 Points avec match direct     : {n_direct:,} ({n_direct/len(df_fire)*100:.1f}%)")
print(f"   🔢 Points nécessitant interpolation : {n_interpolated:,} ({n_interpolated/len(df_fire)*100:.1f}%)")

# ============================================================================
# ÉTAPE 5 : ATTRIBUTION DES PROPRIÉTÉS SOIL
# ============================================================================
print(f"\n[5/7] Attribution des propriétés SOIL...")

# Créer une copie de df_fire pour y ajouter les colonnes SOIL
df_merged = df_fire.copy()

# --- 1. COLONNES CATÉGORIELLES : Plus proche voisin ---
print(f"\n   🔤 Attribution catégorielles (plus proche voisin)...")
closest_indices = indices[:, 0]  # Index du plus proche voisin

for col in categorical_cols:
    df_merged[col] = df_soil.iloc[closest_indices][col].values
    
print(f"      ✓ {len(categorical_cols)} colonnes catégorielles assignées")

# --- 2. COLONNES NUMÉRIQUES : Interpolation IDW ---
print(f"\n   🔢 Interpolation numériques (IDW avec K={K_NEIGHBORS}, power={POWER_IDW})...")

for col in numeric_cols:
    interpolated_values = []
    
    for i in range(len(df_fire)):
        neighbor_indices = indices[i]
        neighbor_distances = distances[i]
        
        # Valeurs des k voisins
        neighbor_values = df_soil.iloc[neighbor_indices][col].values
        
        # Calculer les poids IDW
        weights = 1 / (neighbor_distances + 1e-6) ** POWER_IDW
        weights = weights / weights.sum()
        
        # Moyenne pondérée
        interpolated_value = np.sum(weights * neighbor_values)
        interpolated_values.append(interpolated_value)
    
    df_merged[col] = interpolated_values

print(f"      ✓ {len(numeric_cols)} colonnes numériques interpolées")

# ============================================================================
# ÉTAPE 6 : AJOUTER MÉTADONNÉES
# ============================================================================
print(f"\n[6/7] Ajout des métadonnées...")

# Méthode d'attribution
df_merged['assigned_method'] = np.where(
    direct_match_mask,
    'DIRECT_MATCH',
    'KNN_IDW_INTERPOLATION'
)

# Distance au plus proche voisin
df_merged['knn_distance'] = distances[:, 0]

# Distance en km (approximation : 1° ≈ 111 km)
df_merged['knn_distance_km'] = distances[:, 0] * 111

print(f"      ✓ Métadonnées ajoutées")
print(f"        • assigned_method : type d'attribution")
print(f"        • knn_distance    : distance en degrés")
print(f"        • knn_distance_km : distance en kilomètres")

# ============================================================================
# ÉTAPE 7 : RÉORGANISATION ET SAUVEGARDE
# ============================================================================
print("\n[7/7] Réorganisation des colonnes et sauvegarde...")

# Réorganiser les colonnes
# Ordre : [lat, lon, soil_features, method, distance, class]
all_columns = list(df_merged.columns)

base_cols = ['latitude', 'longitude']
meta_cols = ['assigned_method', 'knn_distance', 'knn_distance_km']
target_col = ['class'] if 'class' in all_columns else []

# Colonnes SOIL (tout sauf les colonnes spéciales)
soil_cols = [col for col in all_columns 
             if col not in base_cols + meta_cols + target_col]

# Ordre final
new_order = base_cols + soil_cols + meta_cols + target_col

# Filtrer les colonnes qui existent réellement
new_order = [col for col in new_order if col in df_merged.columns]
df_merged = df_merged[new_order]

print(f"   ✓ Colonnes réorganisées")
if 'class' in df_merged.columns:
    print(f"     'class' en dernière position")

# Sauvegarder
print(f"\n💾 Sauvegarde des fichiers...")

# 1. CSV
csv_output = os.path.join(output_dir, "fire_soil_merged_knn_idw.csv")
df_merged.to_csv(csv_output, index=False, encoding='utf-8-sig')
print(f"   ✓ CSV : {csv_output}")

# 2. Excel
try:
    excel_output = os.path.join(output_dir, "fire_soil_merged_knn_idw.xlsx")
    df_merged.to_excel(excel_output, index=False, engine='openpyxl')
    print(f"   ✓ Excel : {excel_output}")
except Exception as e:
    print(f"   ⚠️  Excel non créé : {e}")

# 3. Rapport statistiques
stats_output = os.path.join(output_dir, "fusion_statistics.txt")
with open(stats_output, 'w', encoding='utf-8') as f:
    f.write("=" * 80 + "\n")
    f.write("RAPPORT DE FUSION FIRE + SOIL\n")
    f.write("=" * 80 + "\n\n")
    
    f.write("DONNÉES D'ENTRÉE\n")
    f.write(f"  • Points FIRE : {len(df_fire):,}\n")
    f.write(f"  • Points SOIL : {len(df_soil):,}\n\n")
    
    f.write("RÉSULTATS\n")
    f.write(f"  • Total points conservés : {len(df_merged):,} (100%)\n")
    f.write(f"  • Match direct (<{DISTANCE_THRESHOLD}°) : {n_direct:,} ({n_direct/len(df_fire)*100:.1f}%)\n")
    f.write(f"  • Interpolation IDW : {n_interpolated:,} ({n_interpolated/len(df_fire)*100:.1f}%)\n\n")
    
    f.write("PARAMÈTRES\n")
    f.write(f"  • K voisins : {K_NEIGHBORS}\n")
    f.write(f"  • Puissance IDW : {POWER_IDW}\n")
    f.write(f"  • Seuil distance : {DISTANCE_THRESHOLD}° (~{DISTANCE_THRESHOLD*111:.1f} km)\n\n")
    
    f.write("DISTANCES K-NN\n")
    f.write(f"  • Minimale : {distances[:, 0].min():.6f}° (~{distances[:, 0].min()*111:.2f} km)\n")
    f.write(f"  • Maximale : {distances[:, 0].max():.6f}° (~{distances[:, 0].max()*111:.2f} km)\n")
    f.write(f"  • Moyenne  : {distances[:, 0].mean():.6f}° (~{distances[:, 0].mean()*111:.2f} km)\n")
    f.write(f"  • Médiane  : {np.median(distances[:, 0]):.6f}° (~{np.median(distances[:, 0])*111:.2f} km)\n\n")
    
    f.write("COLONNES\n")
    f.write(f"  • Total : {len(df_merged.columns)}\n")
    f.write(f"  • Numériques interpolées : {len(numeric_cols)}\n")
    f.write(f"  • Catégorielles (K-NN) : {len(categorical_cols)}\n")

print(f"   ✓ Rapport : {stats_output}")

# ============================================================================
# RAPPORT FINAL
# ============================================================================
print("\n" + "=" * 80)
print("✅ FUSION TERMINÉE AVEC SUCCÈS")
print("=" * 80)

print(f"\n📊 STATISTIQUES FINALES :")
print(f"   • Total points FIRE d'origine  : {len(df_fire):,}")
print(f"   • Points dans le dataset final : {len(df_merged):,}")
print(f"   • Taux de conservation         : 100.0%")

print(f"\n📍 MÉTHODES D'ATTRIBUTION :")
method_counts = df_merged['assigned_method'].value_counts()
for method, count in method_counts.items():
    pct = count / len(df_merged) * 100
    print(f"   • {method:25s} : {count:6,} points ({pct:5.1f}%)")

print(f"\n📏 DISTANCES K-NN :")
print(f"   • Distance minimale   : {distances[:, 0].min():.6f}° (~{distances[:, 0].min()*111:.2f} km)")
print(f"   • Distance maximale   : {distances[:, 0].max():.6f}° (~{distances[:, 0].max()*111:.2f} km)")
print(f"   • Distance moyenne    : {distances[:, 0].mean():.6f}° (~{distances[:, 0].mean()*111:.2f} km)")
print(f"   • Distance médiane    : {np.median(distances[:, 0]):.6f}° (~{np.median(distances[:, 0])*111:.2f} km)")

print(f"\n🔢 INTERPOLATION IDW :")
print(f"   • Colonnes interpolées : {len(numeric_cols)}")
print(f"   • Colonnes catégorielles : {len(categorical_cols)}")
print(f"   • Nombre de voisins    : {K_NEIGHBORS}")
print(f"   • Puissance IDW        : {POWER_IDW}")

print(f"\n🎯 COLONNES DU DATASET FINAL :")
print(f"   • Total colonnes : {len(df_merged.columns)}")

print(f"\n👀 APERÇU DES DONNÉES (5 premières lignes) :")
print("=" * 80)
display_cols = ['latitude', 'longitude', 'assigned_method', 'knn_distance_km']
if 'class' in df_merged.columns:
    display_cols.append('class')
print(df_merged[display_cols].head())

print(f"\n✨ CARACTÉRISTIQUES :")
print(f"   ✅ 100% des points FIRE conservés")
print(f"   ✅ Match direct pour points proches (<{DISTANCE_THRESHOLD}°)")
print(f"   ✅ Interpolation IDW pour points éloignés")
print(f"   ✅ Catégorielles : K-NN (plus proche voisin)")
print(f"   ✅ Numériques : IDW (interpolation pondérée)")
print(f"   ✅ Traçabilité via métadonnées")
if 'class' in df_merged.columns:
    print(f"   ✅ Colonne 'class' en dernière position")

print("\n" + "=" * 80)
print("🚀 DATASET COMPLET PRÊT POUR LA MODÉLISATION !")
print("=" * 80)

FUSION SPATIALE : FIRE + SOIL (K-NN + IDW)

[1/7] Chargement des données...
   📍 Chargement du dataset FIRE...
      ✓ Fire chargé : 140,165 points

   🌍 Chargement du dataset SOIL...
      ✓ Soil chargé : 25,000 points

   📋 Colonnes FIRE : ['latitude', 'longitude', 'class']...
   📋 Colonnes SOIL : ['longitude', 'latitude', 'COARSE', 'CEC_CLAY', 'ELEC_COND_log']...

[2/7] Identification des types de colonnes SOIL...
   📊 Colonnes numériques (IDW)       : 9
   📋 Colonnes catégorielles (K-NN)   : 0
      Exemples numériques : ['COARSE', 'CEC_CLAY', 'ELEC_COND_log', 'ORG_CARBON_log', 'TEB_log']

[3/7] Application K-NN pour tous les points FIRE...
   🔄 Recherche des 5 plus proches points SOIL...
      ✓ K-NN terminé
      • Distance min (1er voisin) : 0.000083° (~0.01 km)
      • Distance max (1er voisin) : 0.104215° (~11.57 km)
      • Distance moyenne          : 0.023043° (~2.56 km)

[4/7] Classification des points (seuil = 0.1°)...
   📍 Points avec match direct     : 140,164 (100.0%)
 